In [1]:
# Read data from hive table
df_spark = spark.sql("""select * from googleplaystore""")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1600595565618_0010,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
/usr/bin/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/bin/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')

In [2]:
# count records
df_spark.count()

10841

In [3]:
# Check the table schema
df_spark.printSchema()

root
 |-- app: string (nullable = true)
 |-- category: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- reviews: integer (nullable = true)
 |-- size: string (nullable = true)
 |-- installs: string (nullable = true)
 |-- type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- content rating: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- last updated: string (nullable = true)
 |-- current ver: string (nullable = true)
 |-- android ver: string (nullable = true)

In [4]:
# show 5records
df_spark.show(5)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|                 app|      category|rating|reviews|size|   installs|type|price|content rating|              genres|    last updated|       current ver| android ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|  August 1, 2018|             1.2.4|4.0.3 and up|
|Ske

In [5]:
# Select 3 columns
df_spark.select("App", "Category", "Rating").show(5)

+--------------------+--------------+------+
|                 App|      Category|Rating|
+--------------------+--------------+------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|
| Coloring book moana|ART_AND_DESIGN|   3.9|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|
+--------------------+--------------+------+
only showing top 5 rows

In [7]:
# Drop empty rows
df_spark = df_spark.dropna(how='any')

In [9]:
# remove unwanted characters from rows
from pyspark.sql.functions import translate, col
df_spark = df_spark.withColumn('Size', translate('Size', 'Mk', ''))
df_spark = df_spark.withColumn('Installs', translate('Installs', '+,', ''))
df_spark = df_spark.withColumn('Price', translate('Price', '$', ''))

df_spark.show(5)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+--------------------+----------------+------------------+------------+
|                 app|      category|rating|reviews|Size|Installs|type|Price|content rating|              genres|    last updated|       current ver| android ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+--------------------+----------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|  19|   10000|Free|    0|      Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  14|  500000|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 8.7| 5000000|Free|    0|      Everyone|        Art & Design|  August 1, 2018|             1.2.4|4.0.3 and up|
|Sketch - Draw & P...|

In [10]:
# remove records where Size is 'Varies with device'
df_spark = df_spark.where((col("Size") != "Varies with device" ))

In [12]:
# covert data types
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType

df_spark = df_spark.withColumn("Installs", df_spark["Installs"].cast(IntegerType()))
df_spark = df_spark.withColumn("Rating", df_spark["Rating"].cast(FloatType()))
df_spark = df_spark.withColumn("Size", df_spark["Size"].cast(FloatType()))
df_spark = df_spark.withColumn("Price", df_spark["Price"].cast(FloatType()))

df_spark.printSchema()

root
 |-- app: string (nullable = true)
 |-- category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- reviews: integer (nullable = true)
 |-- Size: float (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- content rating: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- last updated: string (nullable = true)
 |-- current ver: string (nullable = true)
 |-- android ver: string (nullable = true)

In [13]:
# Rename columns
df_spark = df_spark.withColumnRenamed('Size', 'Size(M)')
df_spark = df_spark.withColumnRenamed('Installs', 'Installs(+)')
df_spark = df_spark.withColumnRenamed('Price', 'Price($)')
df_spark.show(5)

+--------------------+--------------+------+-------+-------+-----------+----+--------+--------------+--------------------+----------------+------------------+------------+
|                 app|      category|Rating|reviews|Size(M)|Installs(+)|type|Price($)|content rating|              genres|    last updated|       current ver| android ver|
+--------------------+--------------+------+-------+-------+-----------+----+--------+--------------+--------------------+----------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   19.0|      10000|Free|     0.0|      Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|   14.0|     500000|Free|     0.0|      Everyone|Art & Design;Pret...|January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|    8.7|    5000000|Free|     0.0|      Everyone|        Art & Design|  August 1, 2018| 

In [14]:
# Remove unwanted columns
df_spark = df_spark.drop("Reviews", "Genres", "Current Ver")
df_spark.show(5)

+--------------------+--------------+------+-------+-----------+----+--------+--------------+----------------+------------+
|                 app|      category|Rating|Size(M)|Installs(+)|type|Price($)|content rating|    last updated| android ver|
+--------------------+--------------+------+-------+-----------+----+--------+--------------+----------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|   19.0|      10000|Free|     0.0|      Everyone| January 7, 2018|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|   14.0|     500000|Free|     0.0|      Everyone|January 15, 2018|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|    8.7|    5000000|Free|     0.0|      Everyone|  August 1, 2018|4.0.3 and up|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5|   25.0|   50000000|Free|     0.0|          Teen|    June 8, 2018|  4.2 and up|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    2.8|     100000|Free|     0.0|      Everyone|   June 20, 2018|  4.4 and up|
+-------

In [15]:
# filter records - show only where rating is Teen
df_spark.filter(df_spark["Content Rating"] == "Teen").show(10)

+--------------------+-------------------+------+-------+-----------+----+--------+--------------+--------------+------------+
|                 app|           category|Rating|Size(M)|Installs(+)|type|Price($)|content rating|  last updated| android ver|
+--------------------+-------------------+------+-------+-----------+----+--------+--------------+--------------+------------+
|Sketch - Draw & P...|     ART_AND_DESIGN|   4.5|   25.0|   50000000|Free|     0.0|          Teen|  June 8, 2018|  4.2 and up|
|Tattoo Name On My...|     ART_AND_DESIGN|   4.2|   20.0|   10000000|Free|     0.0|          Teen| April 2, 2018|  4.1 and up|
|     I Creative Idea|     ART_AND_DESIGN|   4.7|    4.2|      10000|Free|     0.0|          Teen|April 27, 2018|  4.1 and up|
|Android Auto - Ma...|  AUTO_AND_VEHICLES|   4.2|   16.0|   10000000|Free|     0.0|          Teen| July 11, 2018|  5.0 and up|
|Selfie Camera Pho...|             BEAUTY|   4.1|   30.0|      50000|Free|     0.0|          Teen| July 24, 201

In [16]:
# show only where rating is greater than 4.2
df_spark.filter(df_spark["Rating"] > 4.2).show(10)

+--------------------+--------------+------+-------+-----------+----+--------+--------------+------------------+------------+
|                 app|      category|Rating|Size(M)|Installs(+)|type|Price($)|content rating|      last updated| android ver|
+--------------------+--------------+------+-------+-----------+----+--------+--------------+------------------+------------+
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|    8.7|    5000000|Free|     0.0|      Everyone|    August 1, 2018|4.0.3 and up|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5|   25.0|   50000000|Free|     0.0|          Teen|      June 8, 2018|  4.2 and up|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    2.8|     100000|Free|     0.0|      Everyone|     June 20, 2018|  4.4 and up|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    5.6|      50000|Free|     0.0|      Everyone|    March 26, 2017|  2.3 and up|
|Garden Coloring Book|ART_AND_DESIGN|   4.4|   33.0|    1000000|Free|     0.0|      Everyone|September 20, 2017|  3.0 

In [17]:
# show records where price between 1,3
df_spark.filter(df_spark["Price($)"].between(1, 3)).show(10)

+---+--------+------+-------+-----------+----+--------+--------------+------------+-----------+
|app|category|Rating|Size(M)|Installs(+)|type|Price($)|content rating|last updated|android ver|
+---+--------+------+-------+-----------+----+--------+--------------+------------+-----------+
+---+--------+------+-------+-----------+----+--------+--------------+------------+-----------+

In [18]:
# show where version starts with 4
df_spark.filter(df_spark["Android Ver"].startswith("4")).show(10)

+--------------------+--------------+------+-------+-----------+----+--------+--------------+----------------+------------+
|                 app|      category|Rating|Size(M)|Installs(+)|type|Price($)|content rating|    last updated| android ver|
+--------------------+--------------+------+-------+-----------+----+--------+--------------+----------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|   19.0|      10000|Free|     0.0|      Everyone| January 7, 2018|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|   14.0|     500000|Free|     0.0|      Everyone|January 15, 2018|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|    8.7|    5000000|Free|     0.0|      Everyone|  August 1, 2018|4.0.3 and up|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5|   25.0|   50000000|Free|     0.0|          Teen|    June 8, 2018|  4.2 and up|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    2.8|     100000|Free|     0.0|      Everyone|   June 20, 2018|  4.4 and up|
|Smoke E

In [19]:
# show where Last Updated endswith 2018
df_spark.filter(df_spark["Last Updated"].endswith("2018")).show(10)

+--------------------+--------------+------+-------+-----------+----+--------+--------------+----------------+------------+
|                 app|      category|Rating|Size(M)|Installs(+)|type|Price($)|content rating|    last updated| android ver|
+--------------------+--------------+------+-------+-----------+----+--------+--------------+----------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|   19.0|      10000|Free|     0.0|      Everyone| January 7, 2018|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|   14.0|     500000|Free|     0.0|      Everyone|January 15, 2018|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|    8.7|    5000000|Free|     0.0|      Everyone|  August 1, 2018|4.0.3 and up|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5|   25.0|   50000000|Free|     0.0|          Teen|    June 8, 2018|  4.2 and up|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    2.8|     100000|Free|     0.0|      Everyone|   June 20, 2018|  4.4 and up|
|Smoke E

In [20]:
# show basic statistics
df_spark.describe().show()

+-------+--------------------+--------------+------------------+------------------+-------------------+----+--------+---------------+-----------------+------------------+
|summary|                 app|      category|            Rating|           Size(M)|        Installs(+)|type|Price($)| content rating|     last updated|       android ver|
+-------+--------------------+--------------+------------------+------------------+-------------------+----+--------+---------------+-----------------+------------------+
|  count|                7150|          7150|              7150|              7150|               7150|7150|    7150|           7150|             7150|              7150|
|   mean|                null|          null| 4.167468527163659|35.993566433316346|  9092723.417622378|null|     0.0|           null|             null|               NaN|
| stddev|                null|          null|0.5432031714648444| 88.56600332864916|5.207075897104776E7|null|     0.0|           null|            

In [21]:
# show sum of installs based on category
df_cat_sum = df_spark.groupby("Category").agg({'Installs(+)': "sum"})
df_cat_sum.show()

+-------------------+----------------+
|           Category|sum(Installs(+))|
+-------------------+----------------+
|             EVENTS|        10648400|
|             COMICS|        17536100|
|             SPORTS|      1137817665|
|            WEATHER|       138985000|
|      VIDEO_PLAYERS|       891761200|
|  AUTO_AND_VEHICLES|        43769800|
|          PARENTING|        23516010|
|      ENTERTAINMENT|       950860000|
|    PERSONALIZATION|       993094410|
| HEALTH_AND_FITNESS|       868183220|
|   TRAVEL_AND_LOCAL|       356566300|
|BOOKS_AND_REFERENCE|       139766505|
|     FOOD_AND_DRINK|       177517750|
|        PHOTOGRAPHY|      2554164610|
|           BUSINESS|       535520360|
|             FAMILY|      6787590750|
|           SHOPPING|      1504221440|
|     HOUSE_AND_HOME|        74982000|
|               GAME|     29853553455|
|          EDUCATION|       278100000|
+-------------------+----------------+
only showing top 20 rows

In [ ]:
# show sum of installs based on Type
df_size_sum = df_spark.groupby("Type").agg({'Installs(+)': "sum"})
df_size_sum .show()

In [23]:
# show count of version
df_android_count = df_spark.groupby("Android Ver").count().sort(col("count").desc())
df_android_count.show()

+------------------+-----+
|       Android Ver|count|
+------------------+-----+
|        4.1 and up| 1832|
|      4.0.3 and up| 1125|
|        4.0 and up| 1032|
|        4.4 and up|  770|
|        2.3 and up|  503|
|        5.0 and up|  463|
|        4.2 and up|  303|
|      2.3.3 and up|  206|
|        3.0 and up|  183|
|        4.3 and up|  183|
|        2.2 and up|  166|
|        2.1 and up|   92|
|        1.6 and up|   68|
|Varies with device|   54|
|        6.0 and up|   43|
|        7.0 and up|   26|
|        3.2 and up|   25|
|        2.0 and up|   22|
|        5.1 and up|   14|
|        1.5 and up|   11|
+------------------+-----+
only showing top 20 rows

In [35]:
# show where type is paid
df_app_paid = df_spark.filter(df_spark["Type"] == "Free")
df_app_paid.show()

+--------------------+--------------+------+-------+-----------+----+--------+--------------+------------------+------------+
|                 app|      category|Rating|Size(M)|Installs(+)|type|Price($)|content rating|      last updated| android ver|
+--------------------+--------------+------+-------+-----------+----+--------+--------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|   19.0|      10000|Free|     0.0|      Everyone|   January 7, 2018|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|   14.0|     500000|Free|     0.0|      Everyone|  January 15, 2018|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|    8.7|    5000000|Free|     0.0|      Everyone|    August 1, 2018|4.0.3 and up|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5|   25.0|   50000000|Free|     0.0|          Teen|      June 8, 2018|  4.2 and up|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    2.8|     100000|Free|     0.0|      Everyone|     June 20, 2018|  4.4 

In [37]:
# filter where category = 'ART_AND_DESIGN' size < 2 sort descending
df_app_paid = df_app_paid.filter((col("category") == "ART_AND_DESIGN") & (col("size(M)") < 10 )).sort(col("Rating").desc())
df_app_paid.show()

+--------------------+--------------+------+-------+-----------+----+--------+--------------+-----------------+------------+
|                 app|      category|Rating|Size(M)|Installs(+)|type|Price($)|content rating|     last updated| android ver|
+--------------------+--------------+------+-------+-----------+----+--------+--------------+-----------------+------------+
|Harley Quinn wall...|ART_AND_DESIGN|   4.8|    6.0|      10000|Free|     0.0|      Everyone|   April 25, 2018|  3.0 and up|
|   Cardi B Wallpaper|ART_AND_DESIGN|   4.8|    3.7|      50000|Free|     0.0|      Everyone| November 1, 2017|  4.0 and up|
|X Launcher: With ...|ART_AND_DESIGN|   4.7|    4.4|     100000|Free|     0.0|      Everyone|    July 30, 2018|  4.1 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|    8.7|    5000000|Free|     0.0|      Everyone|   August 1, 2018|4.0.3 and up|
|Photo Designer - ...|ART_AND_DESIGN|   4.7|    5.5|     500000|Free|     0.0|      Everyone|    July 31, 2018|  4.1 and up|
